In [1]:
!pip install gdown

In [2]:
# 1. 确保 gdown 已安装
!pip install gdown

# 2. 定义下载参数
FILE_ID = '1QlAdzrHpfBIOZ6SK78yHF2i1u6tikmBc'
OUTPUT_FILENAME = 'SECOND_dataset.zip'
DOWNLOAD_DIR = '/content/SECOND_data'

# 3. 创建目标文件夹
import os
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)

OUTPUT_PATH = os.path.join(DOWNLOAD_DIR, OUTPUT_FILENAME)

print(f"开始下载 SECOND 数据集到: {OUTPUT_PATH}")

# 4. 执行下载
!gdown --id $FILE_ID -O $OUTPUT_PATH

# 5. 解压文件 (假设它是一个 zip 文件)
print("\n开始解压文件...")
!unzip -q $OUTPUT_PATH -d $DOWNLOAD_DIR

print("\n🎉 SECOND 数据集下载和解压完成！")
print(f"数据现在位于 {DOWNLOAD_DIR} 目录下，请检查其内部结构。")

# 可选：检查解压后的文件列表
# print("目录内容预览:")
# !ls -F $DOWNLOAD_DIR

开始下载 SECOND 数据集到: /content/SECOND_data/SECOND_dataset.zip
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1QlAdzrHpfBIOZ6SK78yHF2i1u6tikmBc
From (redirected): https://drive.google.com/uc?id=1QlAdzrHpfBIOZ6SK78yHF2i1u6tikmBc&confirm=t&uuid=77e9993b-603c-4e1d-a7e9-844eea0b125e
To: /content/SECOND_data/SECOND_dataset.zip
100% 2.41G/2.41G [00:21<00:00, 113MB/s] 

开始解压文件...
[/content/SECOND_data/SECOND_dataset.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/SECOND_data/SECOND_dataset.zip or
    

In [3]:
# 安装 unrar 工具
print("安装 unrar 工具...")
!apt-get install unrar

安装 unrar 工具...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [4]:
# 使用 unrar 工具解压文件
RAR_FILE = '/content/SECOND_data/SECOND_dataset.zip'
OUTPUT_DIR = '/content/SECOND_data/'

print(f"开始解压 RAR 文件: {RAR_FILE}...")
# -x: 提取文件，-o+: 覆盖现有文件（如果存在），-d: 目标目录
!unrar x $RAR_FILE $OUTPUT_DIR

print("\n🎉 SECOND 数据集已成功解压！")
print("现在数据已准备好用于变化检测模型的微调。")

流式输出内容被截断，只能显示最后 5000 行内容。
Extracting  /content/SECOND_data/im1/02100.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02101.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02102.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02104.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02105.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02106.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02108.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02117.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02118.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02119.png                            17%  OK 
Extracting  /content/SECOND_data/im1/02121.png       

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint
import glob
import os
import cv2
import sys

# 启用 AUTOTUNE
AUTOTUNE = tf.data.AUTOTUNE

# --- 0. 挂载 Google Drive 和路径配置 ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive 挂载成功。")
except ImportError:
    print("❌ 无法在当前环境中挂载 Google Drive。请确保在 Colab 中运行。")

# LULC 预训练的权重路径 (请检查您的 Drive 路径)
LULC_WEIGHTS_PATH = '/content/drive/MyDrive/LULC_Pretrain_Weights/best_lulc_pretrained_weights.h5'
# SECOND 数据集的根目录 (由前面的解压步骤创建)
SECOND_DATA_ROOT = '/content/SECOND_data/'

# --- 1. 全局配置与参数 ---
IMG_HEIGHT = 512
IMG_WIDTH = 512
NUM_BANDS = 3
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, NUM_BANDS)

# 最终变化检测的输出类别数： 4 类
NUM_CHANGE_CLASSES = 4

# 🚨 Epochs 安全策略：10 个 Epoch
EPOCHS = 25
BATCH_SIZE = 4
LEARNING_RATE = 1e-5

# T1 和 T2 图像/标签路径
IM1_DIR = os.path.join(SECOND_DATA_ROOT, 'im1')
IM2_DIR = os.path.join(SECOND_DATA_ROOT, 'im2')
LABEL1_DIR = os.path.join(SECOND_DATA_ROOT, 'label1')
LABEL2_DIR = os.path.join(SECOND_DATA_ROOT, 'label2')

CD_WEIGHTS_PATH = os.path.join(os.path.dirname(LULC_WEIGHTS_PATH), 'best_cd_finetune_weights.h5')


# --- 2. U-Net 辅助函数 (保持不变) ---

def conv_block(input_tensor, num_filters, kernel_size=(3, 3), name_suffix=''):
    x = layers.Conv2D(num_filters, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same', name=f'conv_{num_filters}_a{name_suffix}')(input_tensor)
    x = layers.BatchNormalization(name=f'bn_{num_filters}_a{name_suffix}')(x)
    x = layers.Conv2D(num_filters, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same', name=f'conv_{num_filters}_b{name_suffix}')(x)
    x = layers.BatchNormalization(name=f'bn_{num_filters}_b{name_suffix}')(x)
    return x

def encoder_block(input_tensor, num_filters, name_prefix):
    x = conv_block(input_tensor, num_filters, name_suffix=f'_{name_prefix}')
    p = layers.MaxPooling2D((2, 2), name=f'pool_{num_filters}_{name_prefix}')(x)
    return x, p

def decoder_block(input_tensor, skip_tensor, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
    x = layers.concatenate([x, skip_tensor])
    x = conv_block(x, num_filters)
    return x

# --- 3. Pseudo-Siamese U-Net 模型构建 (已修复命名冲突) ---

def build_pseudo_siamese_unet(input_shape, num_change_classes):
    input_t1 = layers.Input(input_shape, name='input_t1')
    input_t2 = layers.Input(input_shape, name='input_t2')

    c1_t1, p1_t1 = encoder_block(input_t1, 32, 't1')
    c2_t1, p2_t1 = encoder_block(p1_t1, 64, 't1')
    c3_t1, p3_t1 = encoder_block(p2_t1, 128, 't1')
    c4_t1, p4_t1 = encoder_block(p3_t1, 256, 't1')

    c1_t2, p1_t2 = encoder_block(input_t2, 32, 't2')
    c2_t2, p2_t2 = encoder_block(p1_t2, 64, 't2')
    c3_t2, p3_t2 = encoder_block(p2_t2, 128, 't2')
    c4_t2, p4_t2 = encoder_block(p3_t2, 256, 't2')

    # 🚨 命名修正：确保瓶颈层名称唯一
    b_t1 = conv_block(p4_t1, 512, name_suffix='_bottleneck_t1')
    b_t2 = conv_block(p4_t2, 512, name_suffix='_bottleneck_t2')

    bottleneck_diff = layers.Subtract(name='bottleneck_diff')([b_t1, b_t2])

    diff_c4 = layers.Subtract(name='skip_diff_c4')([c4_t1, c4_t2])
    diff_c3 = layers.Subtract(name='skip_diff_c3')([c3_t1, c3_t2])
    diff_c2 = layers.Subtract(name='skip_diff_c2')([c2_t1, c2_t2])
    diff_c1 = layers.Subtract(name='skip_diff_c1')([c1_t1, c1_t2])

    u4 = decoder_block(bottleneck_diff, diff_c4, 256)
    u3 = decoder_block(u4, diff_c3, 128)
    u2 = decoder_block(u3, diff_c2, 64)
    u1 = decoder_block(u2, diff_c1, 32)

    outputs = layers.Conv2D(num_change_classes, (1, 1), activation='softmax', name='change_output')(u1)
    model = Model(inputs=[input_t1, input_t2], outputs=outputs, name='Pseudo_Siamese_CD')
    return model


# --- 4. 权重迁移 (保持不变) ---

def transfer_weights(cd_model, lulc_weights_path):
    if not os.path.exists(lulc_weights_path):
        print(f"❌ 警告：LULC 权重文件不存在于 {lulc_weights_path}")
        return cd_model

    try:
        temp_lulc_model = tf.keras.models.load_model(lulc_weights_path, compile=False)
    except Exception as e:
        print(f"❌ 错误：加载 LULC 权重失败：{e}")
        return cd_model

    print("✅ LULC 预训练权重已加载到临时模型。")

    for lulc_layer in temp_lulc_model.layers:
        weights = lulc_layer.get_weights()
        if not weights: continue

        if lulc_layer.name.startswith(('conv', 'bn', 'pool')) and 't1' not in lulc_layer.name:
            t1_layer_name = lulc_layer.name
            if 'pool' not in lulc_layer.name:
                t1_layer_name = lulc_layer.name.replace('_a', '_a_t1').replace('_b', '_b_t1')
            else:
                 t1_layer_name = lulc_layer.name + '_t1'

            try:
                cd_model.get_layer(t1_layer_name).set_weights(weights)
                cd_model.get_layer(t1_layer_name.replace('_t1', '_t2')).set_weights(weights)
            except (ValueError, KeyError):
                pass

    print("✅ 权重迁移完成：LULC 编码器权重已成功赋给 Pseudo-Siamese 模型的 T1/T2 编码器。")
    del temp_lulc_model
    return cd_model


# --- 5. 变化检测数据加载器 (核心逻辑：已修复 PNG 和路径解码) ---

def create_change_map_label(label1_path, label2_path):
    """
    读取 LULC 标签，计算变化，并将其映射到我们的 4 个目标类别。
    参数 label1_path, label2_path 必须是标准 Python 字符串。
    """
    WATER_CLASS_ID = 4
    VEG_CLASS_IDS = [3, 5]

    # 🚨 使用 cv2 读取 PNG 标签（灰度图）
    L1 = cv2.imread(label1_path, cv2.IMREAD_GRAYSCALE)
    L2 = cv2.imread(label2_path, cv2.IMREAD_GRAYSCALE)

    if L1 is None or L2 is None or L1.shape != L2.shape:
        # 抛出错误以中断 tf.py_function
        raise ValueError(f"LULC 标签加载失败或尺寸不匹配。路径: {label1_path}, {label2_path}")

    CM = np.full_like(L1, 3, dtype=np.int32)

    CM[L1 == L2] = 0
    flood_mask = (L2 == WATER_CLASS_ID) & (L1 != WATER_CLASS_ID)
    CM[flood_mask] = 1
    veg_mask = np.isin(L1, VEG_CLASS_IDS)
    not_veg_mask_l2 = ~np.isin(L2, VEG_CLASS_IDS)
    destruct_mask = veg_mask & not_veg_mask_l2 & (L1 != L2)
    CM[destruct_mask] = 2

    return tf.keras.utils.to_categorical(CM, num_classes=NUM_CHANGE_CLASSES).astype(np.float32)


def load_numpy_arrays(im1_fp_str, im2_fp_str, label1_fp_str, label2_fp_str):
    # 🚨 路径解码修复：在传入 cv2 和 create_change_map_label 之前，显式解码所有 Tensor
    im1_fp_str_decoded = im1_fp_str.numpy().decode('utf-8')
    im2_fp_str_decoded = im2_fp_str.numpy().decode('utf-8')
    label1_fp_str_decoded = label1_fp_str.numpy().decode('utf-8')
    label2_fp_str_decoded = label2_fp_str.numpy().decode('utf-8')

    # 图像加载和归一化 (使用解码后的路径)
    im1 = cv2.cvtColor(cv2.imread(im1_fp_str_decoded, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    im2 = cv2.cvtColor(cv2.imread(im2_fp_str_decoded, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

    # 生成变化标签 (使用解码后的路径)
    Y = create_change_map_label(label1_fp_str_decoded, label2_fp_str_decoded)

    return im1, im2, Y

@tf.function(input_signature=[tf.TensorSpec(None, tf.string), tf.TensorSpec(None, tf.string), tf.TensorSpec(None, tf.string), tf.TensorSpec(None, tf.string)])
def load_cd_data_wrapper(im1_fp, im2_fp, label1_fp, label2_fp):

    X1, X2, Y = tf.py_function(
        load_numpy_arrays, [im1_fp, im2_fp, label1_fp, label2_fp], [tf.float32, tf.float32, tf.float32]
    )
    # 确保输出的形状是固定的，否则 tf.data.Dataset 会报错
    X1.set_shape(INPUT_SHAPE)
    X2.set_shape(INPUT_SHAPE)
    Y.set_shape((IMG_HEIGHT, IMG_WIDTH, NUM_CHANGE_CLASSES))
    return (X1, X2), Y


def create_cd_dataset(batch_size, is_training):
    # 收集文件路径，逻辑不变
    img_files = glob.glob(os.path.join(IM1_DIR, '*.png'))
    patch_paths = []

    for img_path_t1 in img_files:
        file_id = os.path.basename(img_path_t1)
        path_t2 = os.path.join(IM2_DIR, file_id)
        path_l1 = os.path.join(LABEL1_DIR, file_id) # .png
        path_l2 = os.path.join(LABEL2_DIR, file_id) # .png

        if os.path.exists(path_t2) and os.path.exists(path_l1) and os.path.exists(path_l2):
             patch_paths.append((img_path_t1, path_t2, path_l1, path_l2))

    if not patch_paths:
        raise ValueError("未能找到匹配的 T1/T2/L1/L2 图像对，请检查路径和文件格式。")

    np.random.shuffle(patch_paths)
    split_idx = int(len(patch_paths) * 0.95)

    current_paths = patch_paths[:split_idx] if is_training else patch_paths[split_idx:]

    paths_tuple = tuple([p[i] for p in current_paths] for i in range(4))
    dataset = tf.data.Dataset.from_tensor_slices(paths_tuple)

    if is_training:
        dataset = dataset.shuffle(buffer_size=min(len(current_paths), 5000))

    dataset = dataset.map(load_cd_data_wrapper, num_parallel_calls=tf.data.AUTOTUNE)

    # 确保 map 和 batch 正常工作
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset, len(current_paths)


# --- 6. 主运行函数 ---

def run_change_detection_finetuning():
    print("\n--- 阶段 I: 数据集准备 (SECOND Changes) ---")

    try:
        train_dataset, num_train = create_cd_dataset(BATCH_SIZE, is_training=True)
        val_dataset, num_val = create_cd_dataset(BATCH_SIZE, is_training=False)

        print(f"训练样本总数: {num_train}, 验证样本总数: {num_val}")
        if num_train == 0 or num_val == 0:
            print("❌ 错误：训练集或验证集样本数为零。请检查文件是否存在。")
            return

    except ValueError as e:
        print(f"❌ 数据集加载失败: {e}")
        return

    print("\n--- 阶段 II: 模型构建与权重迁移 ---")

    cd_model = build_pseudo_siamese_unet(INPUT_SHAPE, NUM_CHANGE_CLASSES)

    cd_model = transfer_weights(cd_model, LULC_WEIGHTS_PATH)

    # 冻结编码器层
    for layer in cd_model.layers:
        if 't1' in layer.name or 't2' in layer.name or 'bottleneck' in layer.name:
            layer.trainable = False

    cd_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=NUM_CHANGE_CLASSES)]
    )

    # 模型检查点保存到 Drive
    cd_checkpoint_callback = ModelCheckpoint(
        filepath=CD_WEIGHTS_PATH, monitor='val_mean_io_u', mode='max', save_best_only=True, verbose=1
    )
    print(f"变化检测权重将保存到 Drive: {CD_WEIGHTS_PATH}")

    print(f"\n--- 阶段 III: 开始变化检测微调 (目标: {EPOCHS} 个 Epoch, 只训练解码器) ---")

    cd_model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        callbacks=[cd_checkpoint_callback],
        verbose=1
    )

    print("\n🎉 变化检测微调完成！")

if __name__ == '__main__':
    run_change_detection_finetuning()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 挂载成功。

--- 阶段 I: 数据集准备 (SECOND Changes) ---
训练样本总数: 2819, 验证样本总数: 149

--- 阶段 II: 模型构建与权重迁移 ---
✅ LULC 预训练权重已加载到临时模型。
✅ 权重迁移完成：LULC 编码器权重已成功赋给 Pseudo-Siamese 模型的 T1/T2 编码器。
变化检测权重将保存到 Drive: /content/drive/MyDrive/LULC_Pretrain_Weights/best_cd_finetune_weights.h5

--- 阶段 III: 开始变化检测微调 (目标: 25 个 Epoch, 只训练解码器) ---
Epoch 1/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.3303 - loss: 1.5226 - mean_io_u: 0.3750
Epoch 1: val_mean_io_u improved from -inf to 0.37500, saving model to /content/drive/MyDrive/LULC_Pretrain_Weights/best_cd_finetune_weights.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 371s 457ms/step - accuracy: 0.3305 - loss: 1.5225 - mean_io_u: 0.3750 - val_accuracy: 0.6322 - val_loss: 1.2974 - val_mean_io_u: 0.3750
Epoch 2/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.6814 - loss: 1.2484 - mean_io_u: 0.3750
Epoch 2: val_mean_io_u improved from 0.37500 to 0.37500, saving model to /content/drive/MyDrive/LULC_Pretrain_Weights/best_cd_finetune_weights.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 284s 402ms/step - accuracy: 0.6814 - loss: 1.2484 - mean_io_u: 0.3750 - val_accuracy: 0.7692 - val_loss: 1.1988 - val_mean_io_u: 0.3750
Epoch 3/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step - accuracy: 0.7671 - loss: 1.1661 - mean_io_u: 0.3750
Epoch 3: val_mean_io_u did not improve from 0.37500
705/705 ━━━━━━━━━━━━━━━━━━━━ 281s 398ms/step - accuracy: 0.7671 - loss: 1.1661 - mean_io_u: 0.3750 - val_accuracy: 0.7726 - val_loss: 1.2485 - val_mean_io_u: 0.3750
Epoch 4/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step - accuracy: 0.7882 - loss: 1.1015 - mean_io_u: 0.3750
Epoch 4: val_mean_io_u improved from 0.37500 to 0.37500, saving model to /content/drive/MyDrive/LULC_Pretrain_Weights/best_cd_finetune_weights.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 283s 401ms/step - accuracy: 0.7882 - loss: 1.1015 - mean_io_u: 0.3750 - val_accuracy: 0.7977 - val_loss: 1.0832 - val_mean_io_u: 0.3750
Epoch 5/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step - accuracy: 0.7991 - loss: 1.0402 - mean_io_u: 0.3750
Epoch 5: val_mean_io_u did not improve from 0.37500
705/705 ━━━━━━━━━━━━━━━━━━━━ 318s 395ms/step - accuracy: 0.7991 - loss: 1.0402 - mean_io_u: 0.3750 - val_accuracy: 0.7754 - val_loss: 1.2515 - val_mean_io_u: 0.3750
Epoch 6/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.8033 - loss: 1.0022 - mean_io_u: 0.3750
Epoch 6: val_mean_io_u did not improve from 0.37500
705/705 ━━━━━━━━━━━━━━━━━━━━ 279s 395ms/step - accuracy: 0.8034 - loss: 1.0022 - mean_io_u: 0.3750 - val_accuracy: 0.8144 - val_loss: 0.8647 - val_mean_io_u: 0.3750
Epoch 7/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.8095 - loss: 0.9544 - mean_io_u: 0.3750
Epoch 7: val_mean_io_u did not improve from 0.37500
705/705 ━━━━━━━━━━━━━━━━━

705/705 ━━━━━━━━━━━━━━━━━━━━ 321s 396ms/step - accuracy: 0.8159 - loss: 0.8763 - mean_io_u: 0.3750 - val_accuracy: 0.8274 - val_loss: 0.7935 - val_mean_io_u: 0.3750
Epoch 10/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.8237 - loss: 0.8335 - mean_io_u: 0.3750
Epoch 10: val_mean_io_u did not improve from 0.37501
705/705 ━━━━━━━━━━━━━━━━━━━━ 279s 395ms/step - accuracy: 0.8237 - loss: 0.8335 - mean_io_u: 0.3750 - val_accuracy: 0.8299 - val_loss: 0.7923 - val_mean_io_u: 0.3750
Epoch 11/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.8206 - loss: 0.8052 - mean_io_u: 0.3750
Epoch 11: val_mean_io_u did not improve from 0.37501
705/705 ━━━━━━━━━━━━━━━━━━━━ 280s 397ms/step - accuracy: 0.8206 - loss: 0.8052 - mean_io_u: 0.3750 - val_accuracy: 0.7951 - val_loss: 1.2278 - val_mean_io_u: 0.3750
Epoch 12/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.8262 - loss: 0.7710 - mean_io_u: 0.3750
Epoch 12: val_mean_io_u did not improve from 0.37501
705/705 ━━━━━━━━━━━

705/705 ━━━━━━━━━━━━━━━━━━━━ 324s 399ms/step - accuracy: 0.8442 - loss: 0.5182 - mean_io_u: 0.3750 - val_accuracy: 0.8379 - val_loss: 0.5858 - val_mean_io_u: 0.3750
Epoch 24/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step - accuracy: 0.8434 - loss: 0.5062 - mean_io_u: 0.3750
Epoch 24: val_mean_io_u did not improve from 0.37502
705/705 ━━━━━━━━━━━━━━━━━━━━ 297s 422ms/step - accuracy: 0.8434 - loss: 0.5062 - mean_io_u: 0.3750 - val_accuracy: 0.8434 - val_loss: 0.5150 - val_mean_io_u: 0.3750
Epoch 25/25
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.8412 - loss: 0.4933 - mean_io_u: 0.3750
Epoch 25: val_mean_io_u improved from 0.37502 to 0.37508, saving model to /content/drive/MyDrive/LULC_Pretrain_Weights/best_cd_finetune_weights.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 306s 399ms/step - accuracy: 0.8412 - loss: 0.4933 - mean_io_u: 0.3750 - val_accuracy: 0.8426 - val_loss: 0.6376 - val_mean_io_u: 0.3751

🎉 变化检测微调完成！
